<a href="https://colab.research.google.com/github/arelkeselbri/gsi073/blob/main/aula5_visualizacao_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 5: Embeddings com modelo *encoder-only* e visualização

Nesta prática você vai usar embeddings e verificar visualmente a intuição de que conceitos semânticos estão espacialmente próximos nos embeddings.

O roteiro é:

1. **Carregar um modelo *encoder-only* para embeddings**.
2. **Gerar embeddings por sentença** para uma coleção de documentos.
3. **Reduzir a dimensionalidade** e **plotar em 2D** para inspecionar agrupamentos.

---


## 0) Setup do ambiente

Vamos instalar dependências e importar bibliotecas.


In [ ]:
# Se necessário, instale dependências (descomente e rode)
# !pip -q install -U sentence-transformers scikit-learn matplotlib pandas numpy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, FastICA

# Opcional: para separar sentenças de forma simples (sem libs pesadas)
import re


## 1) Mini-dataset
Vamos criar uma coleção pequena de documentos, com **rótulos de tópico** (categorias).
Você pode substituir por textos reais (artigos, notícias, parágrafos de PDFs, etc.).


In [ ]:
docs = [
    {
        "id": "d1",
        "label": "esporte",
        "text": (
            "O time venceu por 2 a 1 no último minuto. "
            "O técnico elogiou a disciplina tática e a força do elenco. "
            "A torcida comemorou a classificação no estádio."
        )
    },
    {
        "id": "d2",
        "label": "esporte",
        "text": (
            "O campeonato começou com jogos equilibrados. "
            "O atacante marcou dois gols e foi o destaque da rodada. "
            "A equipe treinou finalizações durante a semana."
        )
    },
    {
        "id": "d3",
        "label": "economia",
        "text": (
            "A inflação desacelerou neste mês segundo o indicador oficial. "
            "O banco central sinalizou cautela na política monetária. "
            "O mercado reagiu com queda nos juros futuros."
        )
    },
    {
        "id": "d4",
        "label": "economia",
        "text": (
            "A taxa de câmbio variou após o anúncio do pacote fiscal. "
            "Analistas revisaram projeções de crescimento para o próximo ano. "
            "O consumo das famílias ainda mostra recuperação lenta."
        )
    },
    {
        "id": "d5",
        "label": "tecnologia",
        "text": (
            "Um novo modelo de linguagem foi lançado com foco em eficiência. "
            "Pesquisadores discutem alinhamento e segurança em IA. "
            "O treinamento usou dados em múltiplos idiomas usado em olimpíadas."
        )
    },
    {
        "id": "d6",
        "label": "tecnologia",
        "text": (
            "Sistemas de recomendação usam embeddings para medir similaridade. "
            "A compressão de modelos ajuda a reduzir custo de inferência. "
            "A comunidade debate benchmarks e avaliação justa."
        )
    },
]

df_docs = pd.DataFrame(docs)
df_docs


## 2) Quebrar documentos em sentenças

Vamos transformar cada documento em várias sentenças.  
No final, teremos um dataframe com colunas: `doc_id`, `label`, `sentence`.


In [ ]:
def split_sentences_pt(text: str) -> list[str]:
    # Split simples por ., !, ? mantendo robustez básica
    # Para produção, considere spaCy / nltk / blingfire, etc.
    parts = re.split(r'(?<=[\.!\?])\s+', text.strip())
    # Filtra vazios
    return [p.strip() for p in parts if p.strip()]

rows = []
for d in docs:
    for sent in split_sentences_pt(d["text"]):
        rows.append({"doc_id": d["id"], "label": d["label"], "sentence": sent})

df_sents = pd.DataFrame(rows)
df_sents


## 3) Carregar modelo *encoder-only* para embeddings

Para embeddings de sentença, é comum usar modelos já ajustados para similaridade semântica, como os da família **Sentence Transformers**.

Exemplos:
- `sentence-transformers/all-MiniLM-L6-v2` (inglês, ótimo custo/benefício)
- `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` (multilíngue, bom para PT)

Aqui usaremos o multilíngue para funcionar bem em português.


In [ ]:
from sentence_transformers import SentenceTransformer

# MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(MODEL_NAME)

# Embedding de uma frase (teste rápido)
test_vec = model.encode(["Olá mundo!"])
test_vec.shape


## 4) Gerar embeddings para cada sentença

Vamos aplicar `model.encode` em lote (batch) para ser eficiente.


In [ ]:
sentences = df_sents["sentence"].tolist()

# normalize_embeddings=True facilita visualização e similaridade por cosseno
emb = model.encode(
    sentences,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)

emb.shape


Agora vamos guardar esses embeddings no dataframe (uma coluna com array).

In [ ]:
df_sents = df_sents.copy()
df_sents["embedding"] = list(emb)
df_sents.head()


## 5) Redução de dimensionalidade para 2D

Existem diferentes técnicas para reduzir dimensionalidade de embedding.
- **PCA** (linear, rápido; preserva variância global)
- **ICA** (linear, busca componentes estatisticamente independentes)
- **t-SNE** (não-linear, bom para estrutura local; ótimo para visualização)

Frequentemente usa-se **PCA antes do t-SNE** para reduzir a dimensionalidade original para 50 dimensões, o que ajuda em para acelerar o processo e reduzir ruído.

A seguir, vamos observar na prática como isso funciona.


In [ ]:
X = np.vstack(df_sents["embedding"].to_numpy())  # (n_sentences, dim)

# 5.1) PCA direto para 2D
pca2 = PCA(n_components=2, random_state=42)
X_pca2 = pca2.fit_transform(X)

# 5.2) ICA direto para 2D
ica2 = FastICA(n_components=2, random_state=42, max_iter=2000)
X_ica2 = ica2.fit_transform(X)

tsne2 = TSNE(
    n_components=2,
    perplexity=5,           # ajuste conforme o tamanho do dataset (tipicamente 5..50)
    learning_rate="auto",
    init="pca",
    random_state=42,
)
X_tsne2 = tsne2.fit_transform(X)

X_pca2.shape, X_ica2.shape, X_tsne2.shape


## 6) Função de plot 2D (com cores por categoria)

Vamos plotar cada sentença como um ponto 2D.  
**Interpretação esperada:** sentenças do mesmo tópico tendem a ficar mais próximas, mas não é garantido (dataset pequeno!).


In [ ]:
def plot_2d(X2, title: str, df_meta: pd.DataFrame):
    plt.figure(figsize=(7, 5))
    for label, sub in df_meta.groupby("label"):
        idx = sub.index.to_numpy()
        plt.scatter(X2[idx, 0], X2[idx, 1], label=label, alpha=0.85)
    plt.title(title)
    plt.xlabel("dim-1")
    plt.ylabel("dim-2")
    plt.legend()
    plt.grid(True, alpha=0.2)
    plt.show()

plot_2d(X_pca2, "PCA (2D) — embeddings de sentenças", df_sents)
plot_2d(X_ica2, "ICA (2D) — embeddings de sentenças", df_sents)
plot_2d(X_tsne2, "t-SNE (2D) — embeddings de sentenças (com pré-PCA 50D)", df_sents)


## 7) Embedding por documento (agregação)

Às vezes você quer um embedding **por documento**, não por sentença.  
Uma forma simples é fazer **média** dos embeddings das sentenças do documento (pode funcionar razoavelmente).

Depois, repita PCA/t-SNE para visualizar documentos.


In [ ]:
# Agrega por doc: média dos vetores das sentenças
df_doc_emb = (
    df_sents
    .groupby(["doc_id", "label"])["embedding"]
    .apply(lambda vs: np.mean(np.vstack(vs), axis=0))
    .reset_index()
)

X_doc = np.vstack(df_doc_emb["embedding"].to_numpy())

# PCA 2D
X_doc_pca2 = PCA(n_components=2, random_state=42).fit_transform(X_doc)

# t-SNE 2D (sem pré-PCA pois é pequeno)
X_doc_tsne2 = TSNE(
    n_components=2,
    perplexity=3,
    learning_rate="auto",
    init="random",
    random_state=42,
).fit_transform(X_doc)

plot_2d(X_doc_pca2, "PCA (2D) — embeddings por documento (média)", df_doc_emb)
plot_2d(X_doc_tsne2, "t-SNE (2D) — embeddings por documento (média)", df_doc_emb)

df_doc_emb[["doc_id", "label"]]


# Exercícios

## Exercício A — Substitua o dataset
1. Use outro dataset com **50 a 150 documentos**, com **3 a 5 tópicos** (ex.: política, saúde, cultura, tecnologia, esporte).
2. Quebre em sentenças.
3. Gere embeddings e visualize com PCA e t-SNE.
4. Escreva 3 observações:
   - Algum tópico formou cluster claro?
   - Há sentenças “fora do lugar”? Por quê?
   - O que muda ao trocar o modelo (inglês vs multilíngue)?

## Exercício B — Variações de t-SNE
1. Teste `perplexity` em {5, 10, 30}.
2. Teste `init` em {"pca", "random"}.
3. Compare estabilidade visual (semente fixa vs outra).

## Exercício C — PCA antes do t-SNE
1. Rode t-SNE com e sem PCA prévio (50D).
2. Compare tempo e qualidade da separação.

---
